In [2]:
import pandas as pd
import os

## Data Cleaning and Editing

In [13]:
# Navigating to csv file
os.chdir('data')
geogon_od = pd.read_csv('geogon_od.csv')

In [14]:
geogon_od.columns

Index(['Year', 'FIPS', 'State', 'County', 'Deaths', 'Population', 'Crude Rate',
       'Cruder Rate', 'Deathrate per 100', 'Unemployment_rate',
       'Dispense_rate', 'SUMLEV', 'AGEGRP', 'TOT_POP', 'TOT_MALE',
       'TOT_FEMALE', 'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE', 'IA_MALE',
       'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE', 'NA_FEMALE', 'TOM_MALE',
       'TOM_FEMALE', 'NH_MALE', 'NH_FEMALE', 'H_MALE', 'H_FEMALE',
       'Urbanicity', 'Jail Population', 'Incarceration Rate per 100k',
       'PovertyCount', 'PovertyPercentage', 'MedianHHI', 'Latitude',
       'Longitude', 'geometry'],
      dtype='object')

In [15]:
# Recoding urbanicity to ordinal
urban_dict = {'rural' : 1, 'small/mid' : 2, 'suburban': 3, 'urban' : 4}
geogon_od = geogon_od.replace({"Urbanicity": urban_dict})
geogon_od['Urbanicity']

0        2.0
1        4.0
2        2.0
3        2.0
4        2.0
        ... 
15891    NaN
15892    NaN
15893    NaN
15894    NaN
15895    1.0
Name: Urbanicity, Length: 15896, dtype: float64

## Exhaustive Feature Selection

In [31]:
def standardize(raw_data):
    return ((raw_data - np.mean(raw_data, axis = 0)) / np.std(raw_data, axis = 0))

In [65]:
# Filter to years 2010 - 2019
geogon_dec = geogon_od[(geogon_od['Year'] >= 2010) & (geogon_od['Year'] <= 2019)]
geogon_dec = geogon_dec.dropna()
geogon_decy = geogon_dec.copy()

# Dropping variables that we don't plan to include as covariates
# Excluding female variables
geogon_dec = geogon_dec.drop(['Deaths', 'Deathrate per 100', 'Crude Rate', 'Cruder Rate',
                              'FIPS', 'State', 'County', 'geometry', 'SUMLEV', 'AGEGRP',
                              'Longitude', 'Latitude', 'TOT_POP', 'TOT_FEMALE', 'WA_FEMALE',
                              'BA_FEMALE', 'IA_FEMALE', 'AA_FEMALE', 'NA_FEMALE', 'NH_FEMALE',
                              'TOM_FEMALE', 'H_FEMALE'], axis = 1)
geogon_dec

,Year,Population,Unemployment_rate,Dispense_rate,TOT_MALE,WA_MALE,BA_MALE,IA_MALE,AA_MALE,NA_MALE,TOM_MALE,NH_MALE,H_MALE,Urbanicity,Jail Population,Incarceration Rate per 100k,PovertyCount,PovertyPercentage,MedianHHI
5902,2010.0,182265.0,9.9,143.8,89620.0,78717.0,8422.0,661.0,551.0,73.0,1196.0,85166.0,4454.0,2.0,734.54,624.90,24056.0,13.3,47618.0
5903,2010.0,57322.0,9.7,60.1,28385.0,27415.0,424.0,178.0,48.0,35.0,285.0,25801.0,2584.0,3.0,124.25,332.87,9358.0,16.5,42906.0
5904,2010.0,118572.0,11.2,182.5,57096.0,44125.0,11327.0,323.0,375.0,63.0,883.0,54969.0,2127.0,2.0,505.38,639.24,27152.0,23.5,37916.0
5905,2010.0,43643.0,10.1,114.7,21603.0,19011.0,2147.0,109.0,64.0,56.0,216.0,19623.0,1980.0,3.0,175.00,611.12,8813.0,20.4,38553.0
5906,2010.0,38319.0,11.6,145.7,19784.0,12173.0,6618.0,644.0,38.0,13.0,298.0,19331.0,453.0,1.0,185.12,730.49,9135.0,26.1,31365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13750,2018.0,103718.0,3.0,45.4,51691.0,49477.0,677.0,275.0,535.0,28.0,699.0,45581.0,6110.0,1.0,297.00,436.26,10114.0,10.1,64234.0
13751,2018.0,135693.0,2.6,48.2,67275.0,64370.0,871.0,255.0,889.0,31.0,859.0,65042.0,2233.0,3.0,229.00,262.63,6059.0,4.5,75799.0
13752,2018.0,403072.0,2.7,51.1,197976.0,183308.0,3632.0,640.0,7362.0,104.0,2930.0,188228.0,9748.0,3.0,532.00,206.18,19937.0,5.0,87333.0
13753,2018.0,171020.0,2.8,44.6,85991.0,78554.0,2644.0,701.0,2454.0,45.0,1593.0,82301.0,3690.0,2.0,310.00,272.00,16915.0,10.4,57785.0


In [70]:
import numpy as np
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS


# Run exhaustive search with linear regression

y = np.array(geogon_decy['Cruder Rate'])

X = np.array(standardize(geogon_dec))

lr = LinearRegression()

efs = EFS(lr, 
          min_features=5,
          max_features=19,
          scoring='neg_mean_squared_error',
          cv=5,
          print_progress = True,
          n_jobs = 2)

efs.fit(X, y)

print('Best MSE score: %.2f' % efs.best_score_ * (-1))
print('Best subset:', efs.best_idx_)

Features: 518507/519252


Best subset: (0, 2, 3, 8, 10, 11, 14, 15, 16, 18)


In [79]:
# Print the variables that EFS determined is the best subset
geogon_dec.columns[[0, 2, 3, 8, 10, 11, 14, 15, 16, 18]]

Index(['Year', 'Unemployment_rate', 'Dispense_rate', 'AA_MALE', 'TOM_MALE',
       'NH_MALE', 'Jail Population', 'Incarceration Rate per 100k',
       'PovertyCount', 'MedianHHI'],
      dtype='object')

## Calculating VIF

### Manually chosen variables

In [15]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

C:\Users\ashmj\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [14]:
# Determining which variables to include when calculating VIF

# Excluding string variables and ones that have high VIF

noninclude_features = ['Year', 'FIPS', 'State', 'County', 'Latitude',
                       'Longitude', 'geometry', 'Crude Rate', 'TOT_POP', 'AGEGRP', 'Jail Population',
                       'Deathrate per 100', 'SUMLEV', 'TOT_MALE', 'BA_MALE', 'IA_MALE',
                       'AA_MALE', 'NA_MALE', 'TOM_MALE', 'NH_MALE', 'H_MALE', 'TOT_FEMALE',
                       'WA_FEMALE', 'BA_FEMALE', 'IA_FEMALE', 'AA_FEMALE', 'NA_FEMALE', 'TOM_FEMALE',
                       'NH_FEMALE', 'H_FEMALE', 'Population', 'PovertyCount', 'Deaths', 'MedianHHI', 'Cruder Rate']
geogon_num = geogon_od.drop(noninclude_features, axis = 1)
geogon_num = geogon_num.dropna()

In [16]:
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = geogon_num.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(geogon_num.values, i)
                          for i in range(len(geogon_num.columns))]
  
print(vif_data)

                       feature        VIF
0            Unemployment_rate   8.926970
1                Dispense_rate   8.362615
2                      WA_MALE   1.726278
3                   Urbanicity   4.999564
4  Incarceration Rate per 100k   4.045233
5            PovertyPercentage  11.713832


### Best Subset Variables

In [83]:
geogon_best = geogon_dec[['Year', 'Unemployment_rate', 'Dispense_rate', 'AA_MALE', 'TOM_MALE',
       'NH_MALE', 'Jail Population', 'Incarceration Rate per 100k',
       'PovertyCount', 'MedianHHI']]

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = geogon_best.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(geogon_best.values, i)
                          for i in range(len(geogon_best.columns))]
  
print(vif_data)

                       feature        VIF
0                         Year  68.784791
1            Unemployment_rate   8.954940
2                Dispense_rate  10.116277
3                      AA_MALE   5.854949
4                     TOM_MALE  14.211463
5                      NH_MALE  14.546340
6              Jail Population   9.974360
7  Incarceration Rate per 100k   4.811989
8                 PovertyCount  13.589621
9                    MedianHHI  31.760316


Looking at all the VIF scores, there is high multicollinearity within the chosen subset (VIF > 4). The `XX_MALE` columns are likely correlated with each other, since they are all a proportion of the greater population. It seems like `Year` has the highest multicollinearity with other variables because each variable's pattern changes significantly depending on the year. `Jail Population` and `Incarceration Rate per 100k` would also be correlated since they are both variables describing the jail population. `Poverty Count`, `MedianHHI`, and `Unemployment_rate` would also be correlated since they are indicators of county wealth. Some options to decrease the multicollinearity would be to drop certain columns or conduct PCA/LASSO/Ridge Regression.